# User Guide

The purpose of this section is to show how to use the tomobase library. 


## Package Settings

The tomo base library uses a number of global dictionaries in order to store data and functions. By using these functions plugins can be added to the library from external packages or set the package settings on the top level. 

All functions use the logger for verbose messaging to Juypter and the library xp to switch between using the CPU and GPU. 






In [ ]:
from tomobase.globals import  xp, GPUContext, logger

logger.setLevel('INFO')
xp.set_context(context=GPUContext.NUMPY, device=0) # Note this is currently an experimental feature

In [ ]:
from tomobase.globals import TOMOBASE_PROCESSES, TOMOBASE_DATATYPES, TOMOBASE_TILTSCHEMES, TOMOBASE_PHANTOMS

# All libraries have a help function that will list its contents

TOMOBASE_PROCESSES.help()
TOMOBASE_DATATYPES.help()
TOMOBASE_PHANTOMS.help()
TOMOBASE_TILTSCHEMES.help()

### Updating Registers

Any function or class decorated with the correct decorator will be recognized and added by the register on update. Please look in Api Reference/Hooks for more information. 

Note: Currently the registers are built to search tomobase. An update will be made to add a json list of searchable packages.


In [ ]:
TOMOBASE_PROCESSES.update() # calling will update the process registry

An example decorated function is provided below. This function takes in a sinogram and modifies it.

In [ ]:
from tomobase.data import Sinogram
from tomobase.hooks import tomobase_hook_process
from tomobase.registrations.transforms import TOMOBASE_TRANSFORM_CATEGORIES

@tomobase_hook_process(name='Tilt Shift', category=TOMOBASE_TRANSFORM_CATEGORIES.ALIGN.value)
def test_function(sino: Sinogram):
    sino = sino*5
    return sino


## Working With Data Classes

Currently the Tomo Base library supportes three data types.

- Volume
- Sinogram
- Image

### Reading and Writing

Each data class supports reading and writing using from_file or to_file. When no path is specified a Dialog will pop up allowing the selection of a file. Check the API reference on supported file types.

In [ ]:
from tomobase.data import Sinogram, Volume, Image

sinogram = Sinogram.from_file()
#sinogram = Sinogram.from_file("path/to/sinogram/file")

sinogram.to_file()
#sinogram.to_file("path/to/sinogram/file")


Alternatively, using the phantoms module a volume can be generated.

In [ ]:
import tomobase.phantoms as phantoms

volume = phantoms.get_nanocage()

### Data Processing

Please read the API references doc for available data processing functions. This is an example using project to forward project a volume into a sinogram.



In [ ]:
import tomobase.processes as processes
import tomobase.tiltschemes
import numpy as np

tiltschemes = tomobase.tiltschemes.GRS(-70,70,0) # create a tilt scheme
angles = np.array([tiltschemes.get_angle(i) for i in range(70)]) # generate an array of angles from a scheme
sino = processes.project(volume) # create a sinogram from a volume

It is worth noting, that decoration of processes adds two variables to any decorated process 

- inplace (bool): if False the input data is copied prior to processing. Defaults True to save memory
- extend_returns (bool): if the function returns more than a single variable e.g. is a tuple only the first is returned when false. Defaults False. 


In [ ]:
# Sinogram is not copied
sino = processes.alignments.align_sinogram_xcorr(sino)

# Sinogram is copied
sino_copy = processes.alignments.align_sinogram_xcorr(sino, inplace=False)

# Sinogram and offsets after alignment are outputted
sino, offsets = processes.alignments.align_sinogram_xcorr(sino, extend_returns=True)


## Visulaization

If you would like to visualize the data. It is recommended to use [stackview](https://github.com/haesleinhuepf/stackview)

In [ ]:

import stackview

stackview.slice(sino.data) # sinogram
stackview.orthogonal(volume.data) # volume

Docs/scripts in the source code provides more information on how to use 